<a href="https://colab.research.google.com/github/vishnubob/AoC/blob/main/AoC_2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import re
import sys
import collections
import functools
import itertools
import operator
import requests
import numpy as np
import scipy.signal
from tqdm import tqdm
from pprint import pprint

cache_dir = "/content/drive/MyDrive/AoC2022/"

session_fn = f"{cache_dir}/session_id"
with open(session_fn) as fh:
    session_id = fh.read().strip()

def aoc_input(day, split=True):
    url = f"https://adventofcode.com/2022/day/{day}/input"
    cache_fn = f"{cache_dir}/d{day}.dat"
    if not os.path.exists(cache_fn):
        cookies = {'session': session_id}
        resp = requests.get(url, cookies=cookies)
        with open(cache_fn, 'wb') as fh:
            fh.write(resp.content)
    with open(cache_fn) as fh:
        data = fh.read()
    data = data.strip()
    if split:
        data = data.split('\n')
    return data

In [2]:
inp = aoc_input(1)

def parse_input(inp):
    vals = []
    for item in inp:
        if not item:
            yield tuple(vals)
            vals = []
            continue
        vals.append(int(item))
    if vals:
        yield tuple(vals)

# d1p1
sums = list(map(sum, parse_input(inp)))
print(max(sums))
# d1p2
print(sum(sorted(sums, reverse=True)[:3]))


68775
202585


In [3]:
inp = aoc_input(2)

def win_lose_draw(game):
    (x, y) = game
    if ((y + 1) % 3) == x:
        return (y + 1)
    if ((y - 1) % 3) == x:
        return (y + 1 + 6)
    return (y + 1 + 3)

def next_move(game):
    (x, y) = game
    if y == 0:
        return (x, (x - 1) % 3)
    if y == 1:
        return (x, x)
    return (x, (x + 1) % 3)

nums = lambda it: it.split(' ')
inp = [(2 - (ord('C') - ord(x)), 2 - (ord('Z') - ord(y))) for (x, y) in map(nums, inp)]
print(sum(map(win_lose_draw, inp)))
print(sum(map(win_lose_draw, map(next_move, inp))))


10624
14060


In [4]:
inp = aoc_input(3)

packs = [(set(val[:len(val) // 2]) & set(val[len(val) // 2:])) for val in inp]
maps = {chr(ord('a') + x): (x + 1) for x in range(26)}
maps.update({chr(ord('A') + x): (x + 27) for x in range(26)})
print(sum([maps[it[0]] for it in map(list, packs)]))

i = iter(inp)
print(sum([maps[list(it)[0]] for it in [set.intersection(*map(set, it)) for it in (zip(i, i, i))]]))


7428
2650


In [5]:
inp = aoc_input(4)

to_intv = lambda it: tuple(map(int, it.split('-')))
data = [tuple(map(to_intv, line.split(','))) for line in inp]
is_contained = lambda it: (it[0][0] <= it[1][0] and it[0][1] >= it[1][1]) or (it[0][0] >= it[1][0] and it[0][1] <= it[1][1])
print(sum([is_contained(it) for it in data]))
is_overlap = lambda it: (it[0][0] >= it[1][0] and it[0][0] <= it[1][1]) or (it[0][1] >= it[1][0] and it[0][1] <= it[1][1]) or is_contained(it)
print(sum([is_overlap(it) for it in data]))


556
876


In [6]:
space_re = re.compile('\s+')
crate_re = re.compile('[\w]')

inp = """   
    [D] 
[N] [C]  
[Z] [M] [P]
 1   2   3 

move 1 from 2 to 1
move 3 from 1 to 3
move 2 from 2 to 1
move 1 from 1 to 2
"""
inp = inp.split('\n')
inp = aoc_input(5)

moves = []
crates = []
for line in inp:
    if not line.strip():
        continue
    if 'move' in line:
        line = line.split(' ')
        moves.append((int(line[1]), int(line[3]), int(line[5])))
        continue
    if line.startswith(' 1'):
        continue
    crates.append(line[1::4])

max_len = max([len(ln) for ln in crates])
new_crates = []
for ln in crates:
    n_pad = max_len - len(ln)
    ln += ' ' * n_pad
    new_crates.append(ln)
crate_init = new_crates

crates = [list(filter(bool, map(str.strip, st)))[::-1] for st in list(zip(*crate_init))]
for (cnt, from_stack, to_stack) in moves:
    payload = crates[from_stack - 1][-cnt:]
    crates[to_stack - 1] += payload[::-1]
    crates[from_stack - 1] = crates[from_stack - 1][:-cnt]

tops = str.join('', crate_re.findall(str.join('', [it[-1] for it in crates])))
print(tops)

crates = [list(filter(bool, map(str.strip, st)))[::-1] for st in list(zip(*crate_init))]
for (cnt, from_stack, to_stack) in moves:
    payload = crates[from_stack - 1][-cnt:]
    crates[to_stack - 1] += payload
    crates[from_stack - 1] = crates[from_stack - 1][:-cnt]

tops = str.join('', crate_re.findall(str.join('', [it[-1] for it in crates])))
print(tops)

TBVFVDZPN
VLCWHTDSZ


In [7]:
inp = aoc_input(6)

for line in inp:
    if not line.strip():
        continue
    kmers = [(x + 4, len(set(line[x:x+4])) == 4) for x in range(len(line) - 4)]
    for (idx, val) in kmers:
        if val:
            print(idx)
            break

for line in inp:
    if not line.strip():
        continue
    kmers = [(x + 14, len(set(line[x:x+14])) == 14) for x in range(len(line) - 14)]
    for (idx, val) in kmers:
        if val:
            print(idx)
            break

1287
3716


In [8]:
inp = aoc_input(7)

class Directory:
    def __init__(self, name, parent):
        self.name = name
        self.parent = parent
        self.files = {}
        self.dirs = {}
    
    def add_file(self, filename=None, size=None):
        self.files[filename] = size

    def add_directory(self, name=None):
        self.dirs[name] = self.__class__(name=name, parent=self)
    
    def get_directory(self, name=None):
        if name not in self.dirs:
            self.add_directory(name=name)
        return self.dirs[name]
    
    @property
    def local_size(self):
        return sum(self.files.values())

    @property
    def size(self):
        size = 0
        for child in self.dirs.values():
            size += child.size
        return size + self.local_size
    
    def flatten(self):
        for child in self.dirs.values():
            yield from child.flatten()
        yield self

root = Directory(name='/', parent=None)
fs = root

for line in inp:
    parts = line.split(' ')
    if parts[0] == '$':
        if parts[1] == 'cd':
            dir_name = parts[-1]
            if dir_name == '..':
                fs = fs.parent
            else:
                fs = fs.get_directory(dir_name)
        continue
    elif parts[0] == 'dir':
        dir_name = parts[-1]
        fs.add_directory(dir_name)
    else:
        size = int(parts[0])
        filename = parts[1]
        fs.add_file(filename=filename, size=size)

sizes = [node.size for node in root.flatten()]
print(sum([sz for sz in sizes if sz < 1e5]))

total_size = int(7e7)
required_capacity = int(3e7)
available_capacity = total_size - root.size
needed_capacity = required_capacity - available_capacity
deltas = [(node.name, node.size - needed_capacity, node.size) for node in root.flatten() if node.size - needed_capacity > 0]  
deltas = sorted(deltas, key=lambda it: it[1])
print(deltas[0][-1])

1297683
5756764


In [9]:
inp = aoc_input(8)
inp = np.array([list(map(int, line)) for line in inp])

vis_count = (inp.shape[0] - 2) * 4 + 4
for x in range(1, inp.shape[0] - 1):
    for y in range(1, inp.shape[1] - 1):
        val = inp[x][y]
        row = 1 * (inp[x, :] < val)
        col = 1 * (inp[:, y] < val)
        codes = (row[:y], row[y+1:], col[:x], col[x+1:])
        vis_count += int(any([np.sum(code) == len(code) for code in codes]))
print(vis_count)

scores = np.zeros_like(inp)
for x in range(1, inp.shape[0] - 1):
    for y in range(1, inp.shape[1] - 1):
        val = inp[x][y]
        row = 1 * (inp[x, :] < val)
        col = 1 * (inp[:, y] < val)
        codes = (row[:y][::-1], row[y+1:], col[:x][::-1], col[x+1:])
        score = 1
        for code in codes:
            length = 0
            for val in code:
                length += 1
                if not val:
                    break
            score *= length
        scores[x][y] = score

print(np.max(scores))

1827
335580
